All my work related to this challenge at
https://github.com/mulargui/kaggle-house-prices-advanced-regression-techniques

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
####### DATA PREPARATION #####
#load data
dftrain=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv', index_col="Id")
dftest=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv', index_col="Id")

#split train data in features and labels
y = dftrain.SalePrice
x = dftrain.drop('SalePrice', axis=1)

# split test data in features and Ids
Ids = dftest.index
x_predict = dftest

# one data set with all features
X = pd.concat([x,x_predict],keys=[0,1])

In [ ]:
###### FEATURE ENGINEERING #####
#features with null values
X["MSZoning"] = X.MSZoning.fillna('NA')
X["LotFrontage"] = X.LotFrontage.fillna(0)
X["Alley"] = X.Alley.fillna("NA")
X["Utilities"] = X.Utilities.fillna("NA")
X["Exterior1st"] = X.Exterior1st.fillna("NA")
X["Exterior2nd"] = X.Exterior2nd.fillna("NA")
X["MasVnrType"] = X.MasVnrType.fillna("NA")
X["MasVnrArea"] = X.MasVnrArea.fillna(0)
X["BsmtQual"] = X.BsmtQual.fillna("NA")
X["BsmtCond"] = X.BsmtCond.fillna("NA")
X["BsmtExposure"] = X.BsmtExposure.fillna("NA")
X["BsmtFinType1"] = X.BsmtFinType1.fillna("NA")
X["BsmtFinSF1"] = X.BsmtFinSF1.fillna(0)
X["BsmtFinType2"] = X.BsmtFinType2.fillna("NA")
X["BsmtFinSF2"] = X.BsmtFinSF2.fillna(0)
X["BsmtUnfSF"] = X.BsmtUnfSF.fillna(0)
X["TotalBsmtSF"] = X.TotalBsmtSF.fillna(0)
X["Electrical"] = X.Electrical.fillna("NA")
X["BsmtFullBath"] = X.BsmtFullBath.fillna(0)
X["BsmtHalfBath"] = X.BsmtHalfBath.fillna(0)
X["KitchenQual"] = X.KitchenQual.fillna("NA")
X["Functional"] = X.Functional.fillna("NA")
X["FireplaceQu"] = X.FireplaceQu.fillna("NA")
X["GarageType"] = X.GarageType.fillna("NA")
X["GarageYrBlt"] = X.GarageYrBlt.fillna(0)
X["GarageFinish"] = X.GarageFinish.fillna("NA")
X["GarageCars"] = X.GarageCars.fillna(0)
X["GarageArea"] = X.GarageArea.fillna(0)
X["GarageQual"] = X.GarageQual.fillna("NA")
X["GarageCond"] = X.GarageCond.fillna("NA")
X["PoolQC"] = X.PoolQC.fillna("NA")
X["Fence"] = X.Fence.fillna("NA")
X["MiscFeature"] = X.MiscFeature.fillna("NA")
X["SaleType"] = X.SaleType.fillna("NA")

#tranform cathegorical to values
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

X["MSSubClass"] = lb_make.fit_transform(X["MSSubClass"])
X["MSZoning"] = lb_make.fit_transform(X["MSZoning"])
X["Street"] = lb_make.fit_transform(X["Street"])
X["Alley"] = lb_make.fit_transform(X["Alley"])
X["LotShape"] = lb_make.fit_transform(X["LotShape"])
X["LandContour"] = lb_make.fit_transform(X["LandContour"])
X["Utilities"] = lb_make.fit_transform(X["Utilities"])
X["LotConfig"] = lb_make.fit_transform(X["LotConfig"])
X["LandSlope"] = lb_make.fit_transform(X["LandSlope"])
X["Neighborhood"] = lb_make.fit_transform(X["Neighborhood"])
X["Condition1"] = lb_make.fit_transform(X["Condition1"])
X["Condition2"] = lb_make.fit_transform(X["Condition2"])
X["BldgType"] = lb_make.fit_transform(X["BldgType"])
X["HouseStyle"] = lb_make.fit_transform(X["HouseStyle"])
X["RoofStyle"] = lb_make.fit_transform(X["RoofStyle"])
X["RoofMatl"] = lb_make.fit_transform(X["RoofMatl"])
X["Exterior1st"] = lb_make.fit_transform(X["Exterior1st"])
X["Exterior2nd"] = lb_make.fit_transform(X["Exterior2nd"])
X["MasVnrType"] = lb_make.fit_transform(X["MasVnrType"])
X["ExterQual"] = lb_make.fit_transform(X["ExterQual"])
X["ExterCond"] = lb_make.fit_transform(X["ExterCond"])
X["Foundation"] = lb_make.fit_transform(X["Foundation"])
X["BsmtQual"] = lb_make.fit_transform(X["BsmtQual"])
X["BsmtCond"] = lb_make.fit_transform(X["BsmtCond"])
X["BsmtExposure"] = lb_make.fit_transform(X["BsmtExposure"])
X["BsmtFinType1"] = lb_make.fit_transform(X["BsmtFinType1"])
X["BsmtFinType2"] = lb_make.fit_transform(X["BsmtFinType2"])
X["Heating"] = lb_make.fit_transform(X["Heating"])
X["HeatingQC"] = lb_make.fit_transform(X["HeatingQC"])
X["CentralAir"] = lb_make.fit_transform(X["CentralAir"])
X["Electrical"] = lb_make.fit_transform(X["Electrical"])
X["KitchenQual"] = lb_make.fit_transform(X["KitchenQual"])
X["Functional"] = lb_make.fit_transform(X["Functional"])
X["FireplaceQu"] = lb_make.fit_transform(X["FireplaceQu"])
X["GarageType"] = lb_make.fit_transform(X["GarageType"])
X["GarageFinish"] = lb_make.fit_transform(X["GarageFinish"])
X["GarageQual"] = lb_make.fit_transform(X["GarageQual"])
X["GarageCond"] = lb_make.fit_transform(X["GarageCond"])
X["PavedDrive"] = lb_make.fit_transform(X["PavedDrive"])
X["PoolQC"] = lb_make.fit_transform(X["PoolQC"])
X["Fence"] = lb_make.fit_transform(X["Fence"])
X["MiscFeature"] = lb_make.fit_transform(X["MiscFeature"])
X["SaleType"] = lb_make.fit_transform(X["SaleType"])
X["SaleCondition"] = lb_make.fit_transform(X["SaleCondition"])

In [ ]:
###### SPLIT DATA #####
#break it down again in train and test
x,x_predict = X.xs(0),X.xs(1)

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
###### THIS IS THE MODEL SECTION ######
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(x_train, y_train)

# Use the forest's predict method on the validation data
y = rf.predict(x_val)

# Calculate the absolute errors
error = abs(y - y_val)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(error), 2), '.')
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (error / y_val)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
#Feature importance
feature_list=X.columns.values
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
###### PREDICTIONS!!! ######
y_predict = rf.predict(x_predict)

In [ ]:
# Save predictions to a file for submission
output = pd.DataFrame({'Id': Ids,
                       'SalePrice': y_predict})
output.to_csv('submission.csv', index=False)

#create a link to download the file    
from IPython.display import FileLink
FileLink(r'submission.csv')